In [1]:
import numpy as np
import pandas as pd
import math

from data_generator import DataGenerator

In [2]:
# Data generating process
dg = DataGenerator(mu=1.3, p_after_a=0.1, p_after_b=0.1)
data = dg.generate()

In [3]:
data

,player_id,variant,p_sum_before,p_sum_after
0,1,A,0.0,0.0
1,2,A,0.0,0.0
2,3,A,0.0,0.0
3,4,B,0.0,0.0
4,5,A,0.0,0.0
...,...,...,...,...
9995,9996,A,0.0,0.0
9996,9997,B,0.0,0.0
9997,9998,A,0.0,0.0
9998,9999,A,0.0,0.0


Method: Nonparametric bootstrap test

Assumptions:
* number of bootstrap samples: $R=10000$
* Significance level: $\alpha=0.05$
* Test if the mean payment for variant B ($\mu_{B}$) is grater than for the variant A ($\mu_{A}$)

Hypothesis:
* $H_{0}:$ $\mu_{A} = \mu_{B}$
* $H_{1}:$ $\mu_{A} < \mu_{B}$

Decision rule: reject $H_{0}$ if *pvalue < significance level* 

In [4]:
# Nonparametric bootstrap test
# H0: mu_1 = mu_2
# H1: mu_1 < mu_2

R = 10000
x_1 = data.loc[data['variant']=='A', 'p_sum_after'].values
x_2 = data.loc[data['variant']=='B', 'p_sum_after'].values
n_1 = len(x_1)
n_2 = len(x_2)
t = np.mean(x_2) - np.mean(x_1)
x_pooled = np.concatenate((x_1, x_2))

success_count = 0
for _ in range(R):
    y_asterisk_1 = np.random.choice(x_pooled, size=n_1+n_2, replace=True)
    y_asterisk_2 = np.random.choice(x_pooled, size=n_1+n_2, replace=True)
    t_asterisk = np.mean(y_asterisk_2) - np.mean(y_asterisk_1)
    if t_asterisk > t:
        success_count += 1
        
pvalue = (success_count + 1) / (R + 1)
print(f"pvalue: {pvalue}")

pvalue: 0.0006999300069993001


Method: Studentized nonparametric bootstrap test

Assumptions:
* number of bootstrap samples: $R=10000$
* Significance level: $\alpha=0.05$
* Test if the mean payment for variant B ($\mu_{B}$) is grater than for the variant A ($\mu_{A}$)

Hypothesis:
* $H_{0}:$ $\mu_{A} = \mu_{B}$
* $H_{1}:$ $\mu_{A} < \mu_{B}$

Decision rule: reject $H_{0}$ if *pvalue < significance level* 

In [5]:
# Studentized nonparametric bootstrap test
# H0: mu_1 = mu_2
# H1: mu_1 < mu_2

R = 10000
x_1 = data.loc[data['variant']=='A', 'p_sum_after'].values
x_2 = data.loc[data['variant']=='B', 'p_sum_after'].values
n_1 = len(x_1)
n_2 = len(x_2)
y_1 = np.mean(x_1)
y_2 = np.mean(x_2)
s_1 = math.sqrt(sum((x_1 - y_1)**2) / (n_1 - 1))
s_2 = math.sqrt(sum((x_2 - y_2)**2) / (n_2 - 1))
z_0 = (np.mean(x_2)-np.mean(x_1)) / math.sqrt(s_1**2/n_1 + s_2**2/n_2)

success_count = 0
for _ in range(R):
    x_asterisk_1 = np.random.choice(x_1, size=n_1, replace=True)
    x_asterisk_2 = np.random.choice(x_2, size=n_2, replace=True)
    y_asterisk_1 = np.mean(x_asterisk_1)
    y_asterisk_2 = np.mean(x_asterisk_2)
    s_asterisk_1 = math.sqrt(sum((x_asterisk_1 - y_asterisk_1)**2) / (n_1 - 1))
    s_asterisk_2 = math.sqrt(sum((x_asterisk_2 - y_asterisk_2)**2) / (n_2 - 1))
    z_asterisk = (
        (y_asterisk_2 - y_asterisk_1 - (y_2 - y_1)) / 
        (s_asterisk_2**2/n_2 + s_asterisk_1**2/n_1)**(1/2)
    )
    if z_asterisk > z_0:
        success_count += 1
pvalue = (success_count + 1) / (R + 1)
print(f"pvalue: {pvalue}")

pvalue: 0.0086991300869913
